In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords

from ydata_profiling import ProfileReport

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

# your code that triggers the warning goes here

pd.options.mode.chained_assignment = 'warn'  # set it back to the default value

In [2]:
 
# Specifying data types for columns while reading a CSV file
dtype_dict = {
    "number": "str",
    "closed": "str", 
    "case": "str",
    "description": "str",
    "case_type": "category",
    "due_date": "str", 
    "first_response_time": "str", 
    "opened": "str",  
    "account": "category",
    "contact": "category",
    "created_by": "category",
    "business_service": "category",
    "business_service_activity": "category",
    "assigned_to": "category",
    "assignment_group": "category",
    "auto_close": "category",
    "time_worked": "float",
    "reassignment_count": "int",
    "impact": "category",
    "priority": "category",
    "urgency": "category",
    "escalation": "category",
    "comments": "str",
    "case_cause": "category",
    "cause": "str",
    "close_notes": "str",
    "resolution_code": "category",
    "problem": "category",
    "business_percentage": "float",
    "sla_has_breached": "category",
    "duration": "float",
    "openedToClosed": "float",
    "created_by_group": "category"
}

# read csv file into dataframe
df = pd.read_csv('Data/data.csv', dtype = dtype_dict)

# concat short_description and description columns. Handle NaN values
df['short_description'] = df['short_description'].fillna('')
df['description'] = df['description'].fillna('')
df['description'] = df['short_description'].str.cat(df['description'], sep =" ")

# print shape of dataframe 
print(df.shape)
print(df.columns)
display(df)

(11746, 20)
Index(['number', 'account', 'created_by_group', 'business_service',
       'assignment_group', 'impact', 'priority', 'urgency', 'sla_has_breached',
       'case_cause', 'resolution_code', 'opened', 'closed', 'time_worked',
       'openedToClosed', 'business_percentage', 'description',
       'short_description', 'cause', 'close_notes'],
      dtype='object')


,number,account,created_by_group,business_service,assignment_group,impact,priority,urgency,sla_has_breached,case_cause,resolution_code,opened,closed,time_worked,openedToClosed,business_percentage,description,short_description,cause,close_notes
0,CS0011381,Saphir Group Networks AG,Unico,Managed Workplace,Service Desk 1st Level,3 - Low,4 - Low,2 - Medium,TRUE,Unico - Product-error in software or hardware ...,Solved – Fixed by Support/Guidance provided,2020-01-09 15:32:00,2020-05-18 19:10:46,360.0,804.900972,105.62,Herr Steiner - HP Notebook Docking Probleme Ha...,Herr Steiner - HP Notebook Docking Probleme,Workaround mit Windows Energy Einstellungen (K...,Workaround mit Windows Energy Einstellungen (K...
1,CS0011690,GASSMANN Rechtsanwälte AG,Unico,Hosted Desktop,Service Desk 1st Level,3 - Low,5 - Planning,3 - Low,TRUE,Customer - incorrect handling,Solved – Fixed by Support/Guidance provided,2020-01-16 08:19:12,2020-08-19 10:30:06,360.0,804.900972,135.01,E-Mail-Archive TAB im Outlook einrichten (User...,E-Mail-Archive TAB im Outlook einrichten (User...,NaN,Archiv wie gewünscht konfiguriert
2,CS0011797,Siloah AG,Unico,Hosted Desktop,Service Desk 1st Level,3 - Low,3 - Moderate,1 - High,TRUE,Customer - incorrect handling,Solved – Fixed by Support/Guidance provided,2020-01-20 12:26:23,2020-05-18 19:10:48,360.0,804.900972,135.01,Performance am Standort Neuhaus sehr langsam W...,Performance am Standort Neuhaus sehr langsam,NaN,gemäss Telefon mit Sandro Borer ist die Perfor...
3,CS0011896,Interhydro AG,Unico,No Business Service - only best effort service,IT Operations,3 - Low,5 - Planning,3 - Low,TRUE,"Customer - Third party (e.g. Internetprovider,...",Voided/Canceled,2020-01-21 18:18:46,2020-06-23 14:51:27,900.0,804.900972,135.01,Cloudberry Backup: HV002-monthly failed 'Micro...,Cloudberry Backup: HV002-monthly failed,Wechsel auf Altaro,Wechsel auf Altaro
4,CS0012000,Verband Regionaler SD Münchenbuchsee,User,No Business Service - only best effort service,Service Desk 2nd Level,3 - Low,4 - Low,2 - Medium,TRUE,"Customer - Third party (e.g. Internetprovider,...",Inconclusive – Out of scope,2020-01-23 17:09:10,2020-05-18 19:10:49,360.0,804.900972,135.01,Pay Pen funktioniert nicht im Programm SAGE Gu...,Pay Pen funktioniert nicht im Programm SAGE,Sage empfiehlt direkt mit dem Hersteller des P...,Guten Tag Frau Bär\r\r\n\r\r\nVielen Dank für ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11741,CS0076818,Siloah AG,Unico,Anmeldeprobleme auf Hosted Desktop beheben (Pa...,Squad Platform,2 - Medium,3 - Moderate,2 - Medium,FALSE,Customer - Product-error in software or hardwa...,Solved – Fixed by Support/Guidance provided,2024-02-10 07:48:59,2024-02-19 22:30:09,2700.0,230.686111,0.03,"Luana Filippi, Anmeldung VDI nicht möglich Fra...","Luana Filippi, Anmeldung VDI nicht möglich",Probleme mit Citrix Workspace App.,Workaround-Lösung implementiert. Login via Bro...
11742,CS0076821,Siloah AG,Unico,Übrige Störungen und Anfragen Hosted Desktop,Vorortsupport Siloah,2 - Medium,3 - Moderate,2 - Medium,FALSE,Unico - Product-error in software or hardware ...,Solved – Fixed by Support/Guidance provided,2024-02-12 05:57:13,2024-02-19 22:30:10,3240.0,184.549167,4.59,"Nagaratnam Thavaraja - drucken Drucker ""prt-C1...","Nagaratnam Thavaraja - drucken Drucker ""prt-C1...",unbekannt,neustart des Druckers und überprüfung der Netz...
11743,CS0076822,PB Swiss Tools,Unico,Session freigeben in Hosted Desktop,Service Desk 1st Level,2 - Medium,3 - Moderate,2 - Medium,FALSE,Unico - Product-error in software or hardware ...,Solved – Fixed by Support/Guidance provided,2024-02-12 06:06:02,2024-02-19 22:30:10,900.0,184.402222,1.80,Parthiban Nathan - Desktop VDI kann nicht mehr...,Parthiban Nathan - Desktop VDI kann nicht mehr...,"blockierte Session auf dem Server Staus ""Loggi...",Session auf dem Server geschlossen - danach ka...
11744,CS0076849,Siloah AG,Unico,Applikationsstörung (Standard Applikation) beh...,Service Desk 1st Level,2 -

In [3]:
# display Short Description (case), Description, Cause and close_notes
df_text = df[['number', 'description', 'cause', 'close_notes']]

# preprocess df_text columns 

display(df_text)

,number,description,cause,close_notes
0,CS0011381,Herr Steiner - HP Notebook Docking Probleme Ha...,Workaround mit Windows Energy Einstellungen (K...,Workaround mit Windows Energy Einstellungen (K...
1,CS0011690,E-Mail-Archive TAB im Outlook einrichten (User...,NaN,Archiv wie gewünscht konfiguriert
2,CS0011797,Performance am Standort Neuhaus sehr langsam W...,NaN,gemäss Telefon mit Sandro Borer ist die Perfor...
3,CS0011896,Cloudberry Backup: HV002-monthly failed 'Micro...,Wechsel auf Altaro,Wechsel auf Altaro
4,CS0012000,Pay Pen funktioniert nicht im Programm SAGE Gu...,Sage empfiehlt direkt mit dem Hersteller des P...,Guten Tag Frau Bär\r\r\n\r\r\nVielen Dank für ...
...,...,...,...,...
11741,CS0076818,"Luana Filippi, Anmeldung VDI nicht möglich Fra...",Probleme mit Citrix Workspace App.,Workaround-Lösung implementiert. Login via Bro...
11742,CS0076821,"Nagaratnam Thavaraja - drucken Drucker ""prt-C1...",unbekannt,neustart des Druckers und überprüfung der Netz...
11743,CS0076822,Parthiban Nathan - Desktop VDI kann nicht mehr...,"blockierte Session auf dem Server Staus ""Loggi...",Session auf dem Server geschlossen - danach ka...
11744,CS0076849,"Christian Kunz, Fehlender Archiv ordner Nach d...",Wurde beim vorderen Case nicht wieder verknüpf...,Archivorder aus dem Laufwerk H:\Outlook ordner...


In [4]:
def process_captions(data, column):
    data[column] = data[column].apply(lambda x: x.replace('\n', ' '))
    data[column] = data[column].apply(lambda x: x.replace('-', ' '))
    data[column] = data[column].apply(lambda x: x.lower())
    data[column] = data[column].apply(lambda x: re.sub('[^a-zA-ZäÄöÖüÜ\s]', ' ', x))
    return data[column]

# change datatype of columns to string
df_text['description'] = df_text['description'].astype(str).copy()
df_text['cause'] = df_text['cause'].astype(str).copy()
df_text['close_notes'] = df_text['close_notes'].astype(str).copy()

df_text['description'] = process_captions(df_text, 'description').copy()
df_text['cause'] = process_captions(df_text, 'cause').copy()
df_text['close_notes'] = process_captions(df_text, 'close_notes').copy()

display(df_text)

C:\Users\Student\AppData\Local\Temp\ipykernel_24244\4016132248.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['description'] = df_text['description'].astype(str).copy()
C:\Users\Student\AppData\Local\Temp\ipykernel_24244\4016132248.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['cause'] = df_text['cause'].astype(str).copy()
C:\Users\Student\AppData\Local\Temp\ipykernel_24244\4016132248.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,number,description,cause,close_notes
0,CS0011381,herr steiner hp notebook docking probleme ha...,workaround mit windows energy einstellungen k...,workaround mit windows energy einstellungen k...
1,CS0011690,e mail archive tab im outlook einrichten user...,nan,archiv wie gewünscht konfiguriert
2,CS0011797,performance am standort neuhaus sehr langsam w...,nan,gemäss telefon mit sandro borer ist die perfor...
3,CS0011896,cloudberry backup hv monthly failed micro...,wechsel auf altaro,wechsel auf altaro
4,CS0012000,pay pen funktioniert nicht im programm sage gu...,sage empfiehlt direkt mit dem hersteller des p...,guten tag frau bär\r\r \r\r vielen dank für ih...
...,...,...,...,...
11741,CS0076818,luana filippi anmeldung vdi nicht möglich fra...,probleme mit citrix workspace app,workaround lösung implementiert login via bro...
11742,CS0076821,nagaratnam thavaraja drucken drucker prt c ...,unbekannt,neustart des druckers und überprüfung der netz...
11743,CS0076822,parthiban nathan desktop vdi kann nicht mehr...,blockierte session auf dem server staus loggi...,session auf dem server geschlossen danach ka...
11744,CS0076849,christian kunz fehlender archiv ordner nach d...,wurde beim vorderen case nicht wieder verknüpf...,archivorder aus dem laufwerk h outlook ordner...


In [5]:
import requests

stopwords = requests.get("https://raw.githubusercontent.com/stopwords-iso/stopwords-de/master/stopwords-de.txt").text.split("\n")
# Add common words to stopwords. Added dynamically after checking the most common words
stopwords.extend(["nan", "frau", "herr", "name", "ch", "bitte", "und", "die", "das", "ist", "zu", "den", "der", "es", "ein", "sie", 
                        "nicht", "von", "mit", "dem", "sich", "auf", "für", "an", "sind", "des", "wird", "dass", "im", "auch", "als", 
                        "an", "nach", "wie", "aber", "aus", "bei", "durch", "hat", "man", "noch", "einem", "über", "einer", "um", "am", 
                        "ohne", "zwischen", "so", "nur", "zum", "kann", "vor", "dieser", "bis", "habe", "wenn", "sein", "wird", "wurde", 
                        "können", "gegen", "dann", "müssen", "diese", "weil", "welche", "oder", "zwei", "eines", "mehr", "Jahre", "wieder", 
                        "keine", "hallo", "grüsse", "gruss", "vielen", "besten", "dank", "guten", "morgen", "tag", "freundlich", "liebe", 
                        "lieber", "sehr geehrte", "geehrter", "geehrte", "hi", "de", "guten tag", "beste", "herzliche", "liebe grüße", "vielen dank", 
                        "besten dank", "freundliche", "grüße", "danke", "siehe", "tel", "mail", "mehr", "à", "vous", "la", "le", "e", "en", "et", "mon", "je", 
                        "les", "pas", "que", "a", "c", "e", "compt", "est", "une", "il", "the", "at"])
from sklearn.feature_extraction.text import TfidfVectorizer

# Remove all characters after "Freundliche Grüsse" or "Beste Grüsse"
df_text['description'] = df_text['description'].apply(lambda x: x.split('freundliche gr')[0])
df_text['description'] = df_text['description'].apply(lambda x: x.split('beste gr')[0])
df_text['description'] = df_text['description'].apply(lambda x: x.split('vielen dank')[0])
df_text['description'] = df_text['description'].apply(lambda x: x.split('von:')[0])

# Create empty df to store the top 50 words for each column
top_50_words = pd.DataFrame()

for column in df_text.columns[1:]:
    tfidf = TfidfVectorizer(stop_words=stopwords)
    text = tfidf.fit_transform(df_text[column])

    VectorizedText = pd.DataFrame(text.toarray(), columns=tfidf.get_feature_names_out())
    # Add to all columns the column name
    VectorizedText.columns = [column + "_" + col for col in VectorizedText.columns]
    column_sums = VectorizedText.sum(axis=0).sort_values(ascending=False)

    # Select the names of the top 50 columns with the highest sums
    top_50_columns = column_sums.head(15).index
    top_50_words[column] = top_50_columns

    # Subset the original DataFrame to keep only these top 50 columns
    minimized_df = VectorizedText[top_50_columns]

    # Add from the original df the column number to the minimized_df at first position
    minimized_df.insert(0, 'number', df_text['number'])

    # save minimized_df to csv
    minimized_df.to_csv('Data/VectorizedText_' + column + '.csv', index=False)

C:\Users\Student\AppData\Local\Temp\ipykernel_24244\2359770970.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['description'] = df_text['description'].apply(lambda x: x.split('freundliche gr')[0])
C:\Users\Student\AppData\Local\Temp\ipykernel_24244\2359770970.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['description'] = df_text['description'].apply(lambda x: x.split('beste gr')[0])
C:\Users\Student\AppData\Local\Temp\ipykernel_24244\2359770970.py:19: SettingWithCopyWarning: 
A v

In [7]:
# For topic Modelling
top_50_words = pd.DataFrame()
for column in df_text.columns[1:]:
    tfidf = TfidfVectorizer(stop_words=stopwords)
    text = tfidf.fit_transform(df_text[column])

    VectorizedText = pd.DataFrame(text.toarray(), columns=tfidf.get_feature_names_out())
    # Add to all columns the column name
    # VectorizedText.columns = [column + "_" + col for col in VectorizedText.columns]
    column_sums = VectorizedText.sum(axis=0).sort_values(ascending=False)

    # Select the names of the top 50 columns with the highest sums
    top_50_columns = column_sums.head(150).index
    top_50_words[column] = top_50_columns

    # only remain words in df_text that are in top_50_words
    df_text2 = df_text[['number', column]].copy()
    df_text2[column] = df_text2[column].apply(lambda x: ' '.join([word for word in x.split() if word in top_50_words['description'].values]))

    # save minimized_df to csv
    df_text2.to_csv('Data/TopicModel_VectorizedText_' + column + '.csv', index=False)
    
